<a href="https://colab.research.google.com/github/javohir-ergashboyev/DL-projects/blob/main/Email_classification(BERT%2C_Functional_Model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [4]:
spam_df=df[df['Category']=='spam'].copy()
ham_df=df[df['Category']=='ham'].copy()

In [5]:
ham_df_balanced=ham_df.sample(spam_df.shape[0])
ham_df_balanced.shape

(747, 2)

In [21]:
new_df=pd.concat([ham_df_balanced, spam_df])

In [8]:
new_df.shape

(1494, 2)

In [9]:
new_df.sample(5)

,Category,Message
294,ham,Are you this much buzy
4543,spam,FreeMsg Hi baby wow just got a new cam moby. W...
2215,ham,Ok then i'll let him noe later n ask him call ...
198,ham,is your hamster dead? Hey so tmr i meet you at...
4451,ham,"Awesome, be there in a minute"


In [22]:
new_df['Spam']=new_df['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [23]:
new_df.sample(5)

,Category,Message,Spam
3154,ham,I tagged MY friends that you seemed to count a...,0
5190,spam,Our records indicate u maybe entitled to 5000 ...,1
591,spam,For ur chance to win a £250 wkly shopping spre...,1
2779,spam,For ur chance to win a £250 cash every wk TXT:...,1
4623,ham,They finally came to fix the ceiling.,0


In [76]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(new_df['Message'], new_df['Spam'], stratify=new_df['Spam'])

In [77]:
xtrain.head()

2954    URGENT! Your mobile was awarded a £1,500 Bonus...
1880    U have a secret admirer who is looking 2 make ...
4271    Great. Hope you are using your connections fro...
4987                     That's y u haf 2 keep me busy...
1854    I just made some payments so dont have that mu...
Name: Message, dtype: object

In [ ]:
!pip install tensorflow_text

In [30]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [27]:
prep_url='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [31]:
bert_preprocess=hub.KerasLayer(prep_url)
bert_encoder=hub.KerasLayer(encoder_url)

In [47]:
def get_sentences_embedding(s):
  preprocessed_text=bert_preprocess(s)
  return bert_encoder(preprocessed_text)['pooled_output']

In [48]:
get_sentences_embedding([
    'You have 1 new message. Please call 08715205273',
    'K k:) sms chat with me.'
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8470105 , -0.48408228, -0.87783825, ..., -0.65065163,
        -0.7158482 ,  0.8211229 ],
       [-0.76744145, -0.39426097, -0.6388048 , ..., -0.4376562 ,
        -0.631394  ,  0.77767485]], dtype=float32)>

In [52]:
e=get_sentences_embedding([
    'apple',
    'banana',
    'Cr Ronaldo',
    'IPhone'
])

In [53]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9659773]], dtype=float32)

In [55]:
cosine_similarity([e[0]],[e[2]])

array([[0.80613637]], dtype=float32)

In [79]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [80]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_word_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                              

In [81]:

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [82]:
model.fit(xtrain,ytrain, epochs=10)

Epoch 1/10
35/35 [==============================] - 14s 331ms/step - loss: 0.6666 - accuracy: 0.6268
Epoch 2/10
35/35 [==============================] - 12s 337ms/step - loss: 0.5133 - accuracy: 0.8304
Epoch 3/10
35/35 [==============================] - 12s 339ms/step - loss: 0.4432 - accuracy: 0.8589
Epoch 4/10
35/35 [==============================] - 12s 338ms/step - loss: 0.3923 - accuracy: 0.8839
Epoch 5/10
35/35 [==============================] - 12s 336ms/step - loss: 0.3553 - accuracy: 0.8929
Epoch 6/10
35/35 [==============================] - 12s 336ms/step - loss: 0.3314 - accuracy: 0.9000
Epoch 7/10
35/35 [==============================] - 12s 336ms/step - loss: 0.3126 - accuracy: 0.9062
Epoch 8/10
35/35 [==============================] - 12s 335ms/step - loss: 0.2982 - accuracy: 0.9179
Epoch 9/10
35/35 [==============================] - 12s 332ms/step - loss: 0.2749 - accuracy: 0.9134
Epoch 10/10
35/35 [==============================] - 12s 341ms/step - loss: 0.2674 - accura

In [83]:
model.evaluate(xtest, ytest)

12/12 [==============================] - 5s 315ms/step - loss: 0.2806 - accuracy: 0.8877


[0.2805967330932617, 0.8877005577087402]

In [84]:
val=[
    'Hey there!, call 2000000221 and get prizes',
    'Hey Jane, how are you? would you like to go to gym tomorrow?'
]
model.predict(val)

1/1 [==============================] - 1s 1s/step


array([[0.73834985],
       [0.18574597]], dtype=float32)